<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=b6e7ac46fde5f442d49cc2553d465e2febbda5359ad17c578ff3bea30cb24cc0
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.0
    Uninstalling protobuf-6.33.0:
      Successfully uninstalled protobuf-6.33.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-10-30 14:09:57
-------------------
qualified stocks: 88
with latest results: 27
still star stocks: 17
-------------------
Initial Investment:  1.26 C
CY Investment:  1.53 C
Reserve:  1.48 L
Current:  1.45 C
-------------------
Today PnL: -81.28 K (-0.56%)
Current PnL: -18.77 L (-12.3%)
CY Booked + Current PnL: -6.64 L (-4.35%)
-------------------
Total profit:  2.51 L
Total loss:  -21.27 L
-------------------
Total Booked + Current PnL: 20.34 L (16.19%)
Total Booked PnL: 39.11 L (31.13%)
Curr Year Booked PnL: 12.13 L (8.34%)
Prev Year Booked PnL: 26.98 L (21.48%)
Est FTT:  2.31 C
Est FTT PnL: 85.84 L (59.02%)
Deployed:  1.26 C
Current:  1.45 C
CAGR/XIRR %: 8.71%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
59,RELIANCE,1533.0,-7.28,69.0,X-LC,4.55,232253.0,21887.0,6898.0,-1.02,10.40,2.97,13.68,37.0,3.17,1.61,28.65,XY25,NTT,REFINERIES
79,TTKPRESTIG,770.0,107.47,59.0,M-SC,1.28,89355.0,-11422.0,11518.0,-2.07,-11.33,12.89,0.09,245.0,-0.99,0.62,17.39,OX40N,NTT,DURABLES
12,BANKINDIA,149.0,-18.01,69.0,H-MC,16.91,213059.0,33251.0,16107.0,-1.59,18.49,7.56,27.45,88.0,2.06,1.48,58.88,XR,NTT,BANKS
42,ITC,452.0,-36.66,61.0,X-LC,1.28,204646.0,4508.0,16372.0,-0.74,2.25,8.00,10.44,4.0,0.28,1.42,7.94,X40,NTT,FMCG
2,ABBOTINDIA,35195.0,-10.05,39.0,X-MC,2.51,88860.0,-1818.0,16723.0,-0.29,-2.00,18.82,16.44,101.0,-0.11,0.62,17.77,X40,ATH,PHARMA


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
82,VAIBHAVGBL,521.00,81.27,78.0,H-SC,2.62,157125.0,-25650.0,136714.0,8.13,-14.03,87.01,60.77,125.0,-0.19,1.09,40.73,XR,NTT,JEWELLERY
17,CAMS,5211.76,-2.95,56.0,X-SC,0.52,110412.0,8408.0,35520.0,2.28,8.24,32.17,43.06,122.0,0.24,0.77,26.77,X40N,ATH,MISC
72,SYMPHONY,1306.00,-42.85,57.0,M-SC,17.83,121706.0,-49435.0,49376.0,1.61,-28.89,40.57,-0.03,196.0,-1.00,0.85,6.10,OX40N,NTT,DURABLES
65,SFL,1287.00,34.39,55.0,M-SC,5.99,177427.0,-84791.0,154628.0,1.45,-32.34,87.15,26.63,229.0,-0.55,1.23,10.31,XY24,NTT,MISC
56,RAJOOENG,143.10,-41.24,49.0,H-SC,17.11,89190.0,-13310.0,53906.0,1.44,-12.99,60.44,39.61,114.0,-0.25,0.62,8.93,AR,ATH,MISC


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
36,IEX,219.00,-31.78,52.0,H-SC,16.02,194743.0,-5211.0,102435.0,-3.46,-2.61,52.60,48.63,115.0,-0.05,1.35,8.65,XR,NTT,MISC
80,UNIONBANK,163.00,-11.63,53.0,M-LC,9.52,160760.0,19920.0,24564.0,-3.24,14.14,15.28,31.59,66.0,0.81,1.12,44.38,XY24,NTT,BANKS
62,SAIL,228.00,48.36,58.0,M-MC,10.20,234622.0,9660.0,157994.0,-3.06,4.29,67.34,74.53,192.0,0.06,1.63,37.93,XY24,BTT,STEEL
84,VBL,671.64,-14.32,61.0,X-LC,8.50,308377.0,-7465.0,120144.0,-2.44,-2.36,38.96,35.67,5.0,-0.06,2.14,11.16,X40N,ATH,FMCG
54,QUESS,986.00,-46.01,49.0,X-SC,37.96,53874.0,-11132.0,162058.0,-2.11,-17.12,300.81,232.18,198.0,-0.07,0.37,5.08,XY24,NTT,MISC


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
32,HINDZINC,730.22,26.6,47.0,M-LC,11.29,206734.0,1658.0,110913.0,-1.73,0.81,53.65,54.89,52.0,0.01,1.44,25.1,X5K,ATH,METALS


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
79,TTKPRESTIG,770.0,107.47,59.0,M-SC,1.28,89355.0,-11422.0,11518.0,-2.07,-11.33,12.89,0.09,245.0,-0.99,0.62,17.39,OX40N,NTT,DURABLES
46,KANSAINER,340.0,-67.24,52.0,H-SC,2.83,225567.0,-44100.0,80437.0,-0.81,-16.35,35.66,13.47,138.0,-0.55,1.57,14.69,XY24,NTT,PAINTS
18,CERA,9475.0,-22.67,52.0,H-SC,2.49,141105.0,-34798.0,76818.0,0.14,-19.78,54.44,23.89,149.0,-0.45,0.98,21.86,OX40N,NTT,CERAMICS
74,TATAELXSI,9161.0,-22.53,53.0,H-MC,6.44,104880.0,-26955.0,69179.0,-0.33,-20.45,65.96,32.03,98.0,-0.39,0.73,17.47,OX40N,NTT,IT
68,SIS,528.0,2031.61,45.0,H-SC,3.20,86024.0,-25008.0,48612.0,-0.79,-22.52,56.51,21.26,156.0,-0.51,0.60,16.23,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
62,SAIL,228.00,48.36,58.0,M-MC,10.20,234622.0,9660.0,157994.0,-3.06,4.29,67.34,74.53,192.0,0.06,1.63,37.93,XY24,BTT,STEEL
57,RECLTD,446.00,47.87,55.0,M-LC,5.19,207900.0,4730.0,37401.0,-1.96,2.33,17.99,20.74,55.0,0.13,1.44,9.35,XY25,NTT,FINANCE
32,HINDZINC,730.22,26.60,47.0,M-LC,11.29,206734.0,1658.0,110913.0,-1.73,0.81,53.65,54.89,52.0,0.01,1.44,25.10,X5K,ATH,METALS
47,KPIGREEN,731.05,11.32,71.0,H-SC,5.46,132741.0,7444.0,51490.0,-0.93,5.94,38.79,47.03,141.0,0.14,0.92,66.20,MH,ATH,POWER
80,UNIONBANK,163.00,-11.63,53.0,M-LC,9.52,160760.0,19920.0,24564.0,-3.24,14.14,15.28,31.59,66.0,0.81,1.12,44.38,XY24,NTT,BANKS


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
12,BANKINDIA,149.00,-18.01,69.0,H-MC,16.91,213059.0,33251.0,16107.0,-1.59,18.49,7.56,27.45,88.0,2.06,1.48,58.88,XR,NTT,BANKS
86,WHIRLPOOL,2270.00,-37.38,66.0,M-SC,3.74,129720.0,14922.0,79116.0,-0.09,13.00,60.99,81.92,223.0,0.19,0.90,53.57,XR,NTT,DURABLES
47,KPIGREEN,731.05,11.32,71.0,H-SC,5.46,132741.0,7444.0,51490.0,-0.93,5.94,38.79,47.03,141.0,0.14,0.92,66.20,MH,ATH,POWER
37,INDIAMART,4810.62,-53.15,57.0,H-SC,6.68,131016.0,7680.0,123941.0,-0.47,6.23,94.60,106.72,119.0,0.06,0.91,31.06,AR,ATH,MISC
32,HINDZINC,730.22,26.60,47.0,M-LC,11.29,206734.0,1658.0,110913.0,-1.73,0.81,53.65,54.89,52.0,0.01,1.44,25.10,X5K,ATH,METALS


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
75,TATAMOTORS,1065.00,-52.51,19.0,X-LC,20.40,149320.0,-127203.0,237269.0,-0.04,-46.00,158.90,39.81,54.0,-0.54,1.04,5.24,XY24,NTT,AUTO
13,BATAINDIA,2096.00,-40.44,28.0,X-SC,4.94,89125.0,-39545.0,84838.0,-0.30,-30.73,95.19,35.20,219.0,-0.47,0.62,2.30,X40,NTT,FOOTWEAR
24,DMART,5391.45,-18.37,32.0,X-LC,9.66,99792.0,-2794.0,29598.0,-1.69,-2.72,29.66,26.13,38.0,-0.09,0.69,22.15,X40N,ATH,FMCG
45,JSWINFRA,342.00,-20.69,37.0,X-MC,5.24,199125.0,-1222.0,31721.0,-0.41,-0.61,15.93,15.23,178.0,-0.04,1.38,31.44,X40N,NTT,REALTY
31,HINDUNILVR,2922.00,-11.98,38.0,X-LC,7.53,247080.0,-9983.0,45117.0,-0.70,-3.88,18.26,13.67,24.0,-0.22,1.72,15.73,XY25,NTT,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
67,SIEMENS,4671.50,-2.83,46.0,H-LC,1.16,156020.0,-30075.0,77558.0,-0.90,-16.16,49.71,25.51,15.0,-0.39,1.08,14.97,AR,ATH,ELECTRICAL
42,ITC,452.00,-36.66,61.0,X-LC,1.28,204646.0,4508.0,16372.0,-0.74,2.25,8.00,10.44,4.0,0.28,1.42,7.94,X40,NTT,FMCG
53,PGHH,17907.65,-30.55,38.0,X-MC,2.29,201045.0,225.0,67571.0,-0.37,0.11,33.61,33.76,80.0,0.00,1.40,5.32,X40,ATH,FMCG
29,HAVELLS,2069.16,-4.90,52.0,X-MC,2.98,317121.0,-10870.0,117398.0,0.05,-3.31,37.02,32.48,92.0,-0.09,2.20,7.73,X40,ATH,ELECTRICAL
85,VOLTAS,1530.00,0.88,51.0,X-MC,4.20,211725.0,19983.0,17785.0,-0.51,10.42,8.40,19.69,99.0,1.12,1.47,17.93,XY25,NTT,AC


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
13,BATAINDIA,2096.00,-40.44,28.0,X-SC,4.94,89125.0,-39545.0,84838.0,-0.30,-30.73,95.19,35.20,219.0,-0.47,0.62,2.30,X40,NTT,FOOTWEAR
52,PAGEIND,51769.93,-28.03,39.0,X-MC,6.70,81360.0,-2150.0,22179.0,-1.64,-2.57,27.26,23.98,82.0,-0.10,0.57,3.10,X40,ATH,APPARELS
3,ACC,3906.00,-37.64,52.0,X-MC,2.52,186000.0,-51951.0,204600.0,-1.03,-21.83,110.00,64.15,174.0,-0.25,1.29,4.34,XY24,BTT,CEMENT
10,BAJAJHFL,181.50,-14.06,49.0,X-MC,5.89,181204.0,-19302.0,116460.0,-0.14,-9.63,64.27,48.45,90.0,-0.17,1.26,4.49,X40N,ATH,FINANCE
21,COLPAL,3767.14,-3.45,49.0,X-MC,7.65,225570.0,-37795.0,151154.0,-0.53,-14.35,67.01,43.04,84.0,-0.25,1.57,4.72,XY25,ATH,FMCG


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,QUESS,986.00,-46.01,49.0,X-SC,37.96,53874.0,-11132.0,162058.0,-2.11,-17.12,300.81,232.18,198.0,-0.07,0.37,5.08,XY24,NTT,MISC
52,PAGEIND,51769.93,-28.03,39.0,X-MC,6.70,81360.0,-2150.0,22179.0,-1.64,-2.57,27.26,23.98,82.0,-0.10,0.57,3.10,X40,ATH,APPARELS
58,RELAXO,1176.00,-39.59,45.0,X-SC,4.77,82363.0,-62797.0,138724.0,-0.61,-43.26,168.43,52.31,136.0,-0.45,0.57,10.25,X40N,NTT,FOOTWEAR
2,ABBOTINDIA,35195.00,-10.05,39.0,X-MC,2.51,88860.0,-1818.0,16723.0,-0.29,-2.00,18.82,16.44,101.0,-0.11,0.62,17.77,X40,ATH,PHARMA
13,BATAINDIA,2096.00,-40.44,28.0,X-SC,4.94,89125.0,-39545.0,84838.0,-0.30,-30.73,95.19,35.20,219.0,-0.47,0.62,2.30,X40,NTT,FOOTWEAR


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
76,TCS,4389.97,-28.36,52.0,X-LC,10.94,285469.0,-60487.0,127176.0,-0.68,-17.48,44.55,19.28,1.0,-0.48,1.98,5.53,X40,ATH,IT
40,INFY,2275.00,-18.52,54.0,X-LC,5.80,317562.0,4674.0,167006.0,-1.29,1.49,52.59,54.87,3.0,0.03,2.21,9.90,X40,BTT,IT
42,ITC,452.00,-36.66,61.0,X-LC,1.28,204646.0,4508.0,16372.0,-0.74,2.25,8.00,10.44,4.0,0.28,1.42,7.94,X40,NTT,FMCG
84,VBL,671.64,-14.32,61.0,X-LC,8.50,308377.0,-7465.0,120144.0,-2.44,-2.36,38.96,35.67,5.0,-0.06,2.14,11.16,X40N,ATH,FMCG
1,ABB,7934.00,-39.48,55.0,H-LC,5.61,252768.0,-8851.0,128052.0,-0.43,-3.38,50.66,45.57,7.0,-0.07,1.76,7.55,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
7,ASIANTILES,137.00,6966.67,56.0,L-SC,7.80,77563.0,-16247.0,92866.0,-1.16,-17.32,119.73,81.67,269.0,-0.17,0.54,50.20,XR,NTT,CERAMICS
20,COFFEEDAY,80.00,-45.67,48.0,L-SC,28.16,80678.0,-32871.0,72925.0,-0.66,-28.95,90.39,35.27,268.0,-0.45,0.56,97.46,XR,NTT,HOTELS
79,TTKPRESTIG,770.00,107.47,59.0,M-SC,1.28,89355.0,-11422.0,11518.0,-2.07,-11.33,12.89,0.09,245.0,-0.99,0.62,17.39,OX40N,NTT,DURABLES
50,MASFIN,398.61,-19.15,52.0,H-SC,9.16,92100.0,-5880.0,27483.0,1.19,-6.00,29.84,22.05,152.0,-0.21,0.64,33.87,XR,ATH,FINANCE
86,WHIRLPOOL,2270.00,-37.38,66.0,M-SC,3.74,129720.0,14922.0,79116.0,-0.09,13.00,60.99,81.92,223.0,0.19,0.90,53.57,XR,NTT,DURABLES


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
82,VAIBHAVGBL,521.00,81.27,78.0,H-SC,2.62,157125.0,-25650.0,136714.0,8.13,-14.03,87.01,60.77,125.0,-0.19,1.09,40.73,XR,NTT,JEWELLERY
47,KPIGREEN,731.05,11.32,71.0,H-SC,5.46,132741.0,7444.0,51490.0,-0.93,5.94,38.79,47.03,141.0,0.14,0.92,66.20,MH,ATH,POWER
86,WHIRLPOOL,2270.00,-37.38,66.0,M-SC,3.74,129720.0,14922.0,79116.0,-0.09,13.00,60.99,81.92,223.0,0.19,0.90,53.57,XR,NTT,DURABLES
5,ANGELONE,3033.00,16.33,63.0,X-SC,5.56,205533.0,14527.0,43182.0,-0.21,7.61,21.01,30.21,157.0,0.34,1.43,30.07,X40N,NTT,FINANCE
69,SONACOMS,806.63,-32.73,66.0,M-SC,7.83,83668.0,-17491.0,57497.0,-1.04,-17.29,68.72,39.54,202.0,-0.30,0.58,17.98,AR,ATH,AUTO


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.62
1,25,43.98
2,50,74.88


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    43.14
LC    32.26
MC    24.57
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     30.00
X40      16.22
XY25     12.02
XR       11.47
X40N     10.67
AR        7.83
OX40N     6.56
X200      1.77
X5K       1.44
SR        1.07
MH        0.92
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    24.50
X-LC    21.62
X-MC    18.76
M-SC    12.68
M-LC     5.04
H-LC     4.61
X-SC     4.53
H-MC     3.86
M-MC     1.63
L-SC     1.43
L-LC     0.99
L-MC     0.32
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,14.51,-3.11,36.76
IT,12.47,-17.16,79.99
FINANCE,12.13,-8.19,57.23
MISC,6.86,-16.28,80.71
ELECTRICAL,5.81,-7.94,46.21
BANKS,5.61,-6.09,60.96
PAINTS,5.51,-16.43,33.78
INSURANCE,3.78,-1.46,39.29
AUTO,2.76,-44.80,105.23


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3251301.0,22
XR,1235853.0,14
AR,1160763.0,9
X40,821647.0,12
X40N,622448.0,9
OX40N,541331.0,9
XY25,451281.0,8
SR,267511.0,2
X5K,110913.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3304570.0,24
M-SC,1218424.0,15
X-MC,1176484.0,14
X-LC,1024340.0,13
X-SC,528165.0,6
M-LC,283836.0,4
H-LC,274771.0,3
H-MC,261375.0,3
L-SC,255314.0,3


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1154471.0      6
           AR         831114.0      5
           XR         759278.0      7
M-SC       XY24       697891.0      6
X-MC       XY24       574529.0      4
X-LC       X40        451974.0      5
           XY24       303705.0      2
X-MC       X40        284835.0      6
X-SC       X40N       281269.0      4
H-SC       SR         267511.0      2
           OX40N      240706.0      3
H-LC       AR         205610.0      2
X-LC       X40N       192998.0      3
H-MC       XY24       176089.0      1
X-MC       XY25       168939.0      2
L-SC       XR         165791.0      2
X-SC       XY24       162058.0      1
M-MC       XY24       157994.0      1
X-MC       X40N       148181.0      2
M-SC       OX40N      141923.0      4
           XY25       127957.0      1
           XR         126614.0      2
           AR         124039.0      2
M-LC       XR         110958.0      1
           X5K        110913.0      1
L-SC       OX40N       89523.0      1
X-SC       X40         84838.0      1
X-LC       XY25        75663.0      3
H-MC       OX40N       69179.0      1
H-LC       X200        69161.0      1
L-MC       XR          57105.0      1
H-SC       MH          51490.0      1
L-LC       XY25        41321.0      1
M-LC       XY25        37401.0      1
           XY24        24564.0      1
H-MC       XR          16107.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 27.0 seconds
